In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from movement import Akagi, ForwardSimulator

# Set parameter values

In [3]:
num_cells = 16

In [4]:
n = int(num_cells**0.5)
x = np.linspace(-1, 1, n)
y = np.linspace(-1, 1, n)
xv, yv = np.meshgrid(x, y, sparse=False, indexing='ij')

loc = []
for i in range(n):
    for j in range(n):
        loc.append([x[i], y[j]])

loc = np.array(loc)

In [5]:
rad = 0.8
N_init = np.exp(-((loc**2).sum(axis=1)**0.5 - rad)**2)
N_init = (N_init * 1000 * N_init.max()).astype(int)
N_init.reshape((n, n))

array([[642, 878, 878, 642],
       [878, 841, 841, 878],
       [878, 841, 841, 878],
       [642, 878, 878, 642]])

In [6]:
# plt.imshow(N_init.reshape((n, n)))
# plt.colorbar()

In [7]:
s = np.exp((-4 * loc**2).sum(axis=1))
s.reshape((n, n))

array([[3.35462628e-04, 1.17436285e-02, 1.17436285e-02, 3.35462628e-04],
       [1.17436285e-02, 4.11112291e-01, 4.11112291e-01, 1.17436285e-02],
       [1.17436285e-02, 4.11112291e-01, 4.11112291e-01, 1.17436285e-02],
       [3.35462628e-04, 1.17436285e-02, 1.17436285e-02, 3.35462628e-04]])

In [8]:
# plt.imshow(s.reshape((n, n)))
# plt.colorbar()

In [9]:
pi = N_init / N_init.max() / 10
pi.reshape((n, n))

array([[0.07312073, 0.1       , 0.1       , 0.07312073],
       [0.1       , 0.09578588, 0.09578588, 0.1       ],
       [0.1       , 0.09578588, 0.09578588, 0.1       ],
       [0.07312073, 0.1       , 0.1       , 0.07312073]])

In [10]:
# plt.imshow(pi.reshape((n, n)))
# plt.colorbar()

In [11]:
# Array of distances between cells
d = np.zeros((num_cells, num_cells))
for i in range(num_cells):
    for j in range(num_cells):
        d[i, j] = ((loc[i] - loc[j])**2).sum()**0.5

In [12]:
beta = 1
K = 1.5

## Generate fake data

In [13]:
simulator = ForwardSimulator(pi, s, beta, d, K)
simulator.noise_amplitude = 0.0

In [14]:
num_steps = 11
np.random.seed(0)
N, M_true = simulator.simulate(N_init, num_steps)

In [15]:
# fig, ax = plt.subplots()

# cs = ax.imshow(N)
# cb = fig.colorbar(cs)

# ax.set_ylabel('t')
# ax.set_xlabel('cell number')

In [16]:
# fig, ax = plt.subplots()

# ax.plot(N.sum(axis=1), label='total')

# for i in range(N.shape[1]):
#     ax.plot(N[:, i])
    
# ax.set_xlabel('time')
# ax.set_ylabel('population')

# ax.set_yscale('log')

# fig.legend()

## Estimate movement from fake data

In [17]:
from movement import Akagi

In [18]:
a = Akagi(N, d, K)
a.lamda = 10

a.init_M_moving()

a.M_jitter(1/N.max()/10)

In [19]:
M_init = a.M.copy()
pi_init = a.pi.copy()
s_init = a.s.copy()
beta_init = a.beta

Pre-compile some numba functions
This wouldn't be done normally, but is interesting when testing

In [20]:
# from movement import akagi

In [21]:
# %time akagi._cost(M_init, N)
# %time akagi._cost(M_init, N)

In [22]:
term_0_log = a.term_0_log(a.pi)
braces = a.term_1_braces(a.pi, a.s, a.beta, a.d)

# %time a.likelihood(a.M, a.pi, a.s, a.beta, term_0_log=term_0_log, term_1_braces=braces)
# %time a.likelihood(a.M, a.pi, a.s, a.beta, term_0_log=term_0_log, term_1_braces=braces)

In [23]:
%%time
##Use the derivative of the Likelihood function for the optimization of the likelihood function
# %%prun
result = a.exact_inference(1e-4, True)

step #  0 , L =  -6499320.545582052
distances 0.0
distances 0.6666666666666666
distances 1.3333333333333333
distances 2.0
distances 0.6666666666666666
distances 0.9428090415820634
distances 1.4907119849998598
distances 2.1081851067789197
distances 1.3333333333333333
distances 1.4907119849998598
distances 1.8856180831641267
distances 2.4037008503093262
distances 2.0
distances 2.1081851067789197
distances 2.4037008503093262
distances 2.8284271247461903
distances 0.6666666666666666
distances 0.0
distances 0.6666666666666666
distances 1.3333333333333335
distances 0.9428090415820634
distances 0.6666666666666666
distances 0.9428090415820634
distances 1.4907119849998598
distances 1.4907119849998598
distances 1.3333333333333333
distances 1.4907119849998598
distances 1.8856180831641267
distances 2.1081851067789197
distances 2.0
distances 2.1081851067789197
distances 2.4037008503093262
distances 1.3333333333333333
distances 0.6666666666666666
distances 0.0
distances 0.6666666666666667
distances 

/Users/mateja/projects/covid-19/population-data/movement/akagi.py:232: RuntimeWarning: divide by zero encountered in log
  term_2_3 = np.log(pi[m]) + np.log(s[n]) - beta * self.d[m,n] - np.log(sexp.sum(axis=1, where=self.gamma_exc))-np.log(M[l,m,n])


0.6666666666666667
distances 1.4907119849998598
distances 0.9428090415820634
distances 0.6666666666666667
distances 0.9428090415820635
distances 2.4037008503093262
distances 1.8856180831641267
distances 1.4907119849998598
distances 1.3333333333333333
distances 2.1081851067789197
distances 1.4907119849998598
distances 0.9428090415820634
distances 0.6666666666666666
distances 2.0
distances 1.3333333333333335
distances 0.6666666666666667
distances 0.0
distances 2.1081851067789197
distances 1.49071198499986
distances 0.9428090415820635
distances 0.6666666666666667
distances 2.0
distances 2.1081851067789197
distances 2.4037008503093262
distances 2.8284271247461903
distances 1.3333333333333335
distances 1.4907119849998598
distances 1.8856180831641267
distances 2.4037008503093262
distances 0.6666666666666667
distances 0.9428090415820634
distances 1.4907119849998598
distances 2.1081851067789197
distances 0.0
distances 0.6666666666666666
distances 1.3333333333333333
distances 2.0
distances 2.10

distances 0.6666666666666667
distances 0.0
distances 2.1081851067789197
distances 1.4907119849998598
distances 0.9428090415820634
distances 0.6666666666666666
distances 2.4037008503093262
distances 1.8856180831641267
distances 1.4907119849998598
distances 1.3333333333333333
distances 2.8284271247461903
distances 2.4037008503093262
distances 2.1081851067789197
distances 2.0
distances 0.6666666666666666
distances 0.9428090415820634
distances 1.4907119849998598
distances 2.1081851067789197
distances 0.0
distances 0.6666666666666666
distances 1.3333333333333333
distances 2.0
distances 0.6666666666666666
distances 0.9428090415820634
distances 1.4907119849998598
distances 2.1081851067789197
distances 1.3333333333333335
distances 1.4907119849998598
distances 1.8856180831641267
distances 2.4037008503093262
distances 0.9428090415820634
distances 0.6666666666666666
distances 0.9428090415820634
distances 1.4907119849998598
distances 0.6666666666666666
distances 0.0
distances 0.6666666666666666
di

distances 0.0
distances 0.6666666666666666
distances 1.3333333333333335
distances 0.9428090415820634
distances 0.6666666666666667
distances 0.9428090415820634
distances 1.49071198499986
distances 1.8856180831641267
distances 1.4907119849998598
distances 1.3333333333333333
distances 1.4907119849998598
distances 1.4907119849998598
distances 0.9428090415820634
distances 0.6666666666666666
distances 0.9428090415820634
distances 1.3333333333333333
distances 0.6666666666666666
distances 0.0
distances 0.6666666666666667
distances 1.4907119849998598
distances 0.9428090415820634
distances 0.6666666666666667
distances 0.9428090415820635
distances 2.4037008503093262
distances 1.8856180831641267
distances 1.4907119849998598
distances 1.3333333333333333
distances 2.1081851067789197
distances 1.4907119849998598
distances 0.9428090415820634
distances 0.6666666666666666
distances 2.0
distances 1.3333333333333335
distances 0.6666666666666667
distances 0.0
distances 2.1081851067789197
distances 1.490711

distances 0.6666666666666667
distances 0.9428090415820634
distances 1.4907119849998598
distances 2.1081851067789197
distances 1.4907119849998598
distances 1.3333333333333333
distances 1.4907119849998598
distances 1.8856180831641267
distances 0.9428090415820634
distances 0.6666666666666666
distances 0.9428090415820634
distances 1.4907119849998598
distances 0.6666666666666666
distances 0.0
distances 0.6666666666666666
distances 1.3333333333333335
distances 0.9428090415820634
distances 0.6666666666666667
distances 0.9428090415820634
distances 1.49071198499986
distances 1.8856180831641267
distances 1.4907119849998598
distances 1.3333333333333333
distances 1.4907119849998598
distances 1.4907119849998598
distances 0.9428090415820634
distances 0.6666666666666666
distances 0.9428090415820634
distances 1.3333333333333333
distances 0.6666666666666666
distances 0.0
distances 0.6666666666666667
distances 1.4907119849998598
distances 0.9428090415820634
distances 0.6666666666666667
distances 0.94280

distances 1.3333333333333333
distances 1.4907119849998598
distances 1.8856180831641267
distances 2.4037008503093262
distances 0.6666666666666666
distances 0.9428090415820634
distances 1.4907119849998598
distances 2.1081851067789197
distances 0.0
distances 0.6666666666666666
distances 1.3333333333333333
distances 2.0
distances 0.6666666666666667
distances 0.9428090415820634
distances 1.4907119849998598
distances 2.1081851067789197
distances 1.4907119849998598
distances 1.3333333333333333
distances 1.4907119849998598
distances 1.8856180831641267
distances 0.9428090415820634
distances 0.6666666666666666
distances 0.9428090415820634
distances 1.4907119849998598
distances 0.6666666666666666
distances 0.0
distances 0.6666666666666666
distances 1.3333333333333335
distances 0.9428090415820634
distances 0.6666666666666667
distances 0.9428090415820634
distances 1.49071198499986
distances 1.8856180831641267
distances 1.4907119849998598
distances 1.3333333333333333
distances 1.4907119849998598
dis

ValueError: too many axes: 3 (effrank=3), expected rank=1


In [24]:
%%time
##Do not use a derivative of the Likelihood function for the optimization of the likelihood function
# %%prun
result = a.exact_inference(1e-4, False)

step #  0 , L =  -6499320.545582052
Error minimizing M b'STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT'
step #  1 , L =  -1236768.9044375408
Error minimizing M b'STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT'
step #  2 , L =  -826895.2310180293
Error minimizing M b'STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT'
step #  3 , L =  -767417.2923560231
Error minimizing M b'STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT'
step #  4 , L =  -761064.4093173444
Error minimizing M b'STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT'
step #  5 , L =  -760118.4783957916
CPU times: user 37.2 s, sys: 15.8 s, total: 53 s
Wall time: 25.7 s


# Compare output

Does `M` look similar?

In [ ]:
np.abs((M_true - a.M)).sum()/M_true.sum()

In [ ]:
M_true[1][:10, :10]

In [ ]:
np.rint(a.M[0]).astype(int)[:10, :10]

Are there approximately the right number of people in the end?

In [ ]:
np.rint((a.M.sum(axis=2) - N[:-1])).astype(int)

In [ ]:
print(a.likelihood(M_init, pi_init, s_init, beta_init) )
print(a.likelihood(a.M, a.pi, a.s, a.beta))

Have $\pi$,  $s$ and $\beta$ converged well?

In [ ]:
def comp_plot(exact, est, title):
    fig, ax = plt.subplots()
    
    ident = np.linspace(
        min(exact.min(), est.min()),
        max(exact.max(), est.max()),
    )
    ax.plot(ident, ident, alpha=0.5, color='gray')
    
    ax.scatter(
        exact,
        est,
        marker="+"
    )
    
    ax.set_xlabel(r"exact")
    ax.set_ylabel(r"estimated")
    
    ax.set_title(title)
    
    ax.set_aspect('equal')

In [ ]:
comp_plot(pi, a.pi, r"$\pi$")

In [ ]:
comp_plot(s/s.max(), a.s/a.s.max(), r"$s$")

In [ ]:
comp_plot(M_true.flatten(), a.M.flatten(), r"$M$")

In [ ]:
beta

In [ ]:
a.beta

Save some vars

In [ ]:
np.save("N", N)
np.save("d", d)
np.save("K", K)
np.save("beta", beta)
np.save("s", s)
np.save("pi", pi)
np.save("M_true", M_true)